##  - Instal libraries -

In [ ]:
pip install langchain -q
pip install transformers -q
pip install pinecone-client -q
pip install faiss-cpu -q
!pip install SpeechRecognition -q
!pip install pyttsx3 -q

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install transformers -q

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install pinecone-client -q

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [20]:
pip install faiss-cpu -q


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install SpeechRecognition -q


[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: D:\python11\python.exe -m pip install --upgrade pip


In [4]:
!pip install pyttsx3 -q


[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: D:\python11\python.exe -m pip install --upgrade pip


In [4]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool
import os
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.tools.file_management.write import WriteFileTool
from langchain.vectorstores import FAISS

## - Use API KEYS -

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## - Load Dataset -

In [8]:
import json
from langchain_core.documents import Document

# load JSON File
with open(r'D:\Project\RimalAI_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [16]:
from langchain.schema import Document

# Initialize an empty list to store the documents
documents = []

def recursive_extract(content, level=0):
    """
    Helper function to recursively extract nested data structures into a readable format.
    """
    if isinstance(content, dict):
        result = ""
        for key, value in content.items():
            result += f"{'  ' * level}{key}:\n"
            result += recursive_extract(value, level + 1)
        return result
    elif isinstance(content, list):
        result = ""
        for item in content:
            result += recursive_extract(item, level + 1)
        return result
    else:
        return f"{'  ' * level}{content}\n"

# Convert the loaded data into structured documents
for item in data:
    # Extract basic fields
    name = item.get('name', 'No Name')
    description = item.get('description', 'No Description')
    type_of_item = item.get('type', 'No Type')
    id_item = item.get('id', 'No ID')

    # Prepare the main content of the document
    content = f"Name: {name}\nDescription: {description}\n"

    # Extract nested details
    content += "\nDetails:\n"
    content += recursive_extract(item, level=1)

    # Create and append the Document
    documents.append(
        Document(
            page_content=content,
            metadata={
                "type": type_of_item,
                "name": name,
                "id": id_item
            }
        )
    )

# Print documents to verify
for doc in documents:
    print(f"Type: {doc.metadata['type']}, Name: {doc.metadata['name']}, ID: {doc.metadata['id']}, Content: {doc.page_content[:100]}...")

Type: landmark, Name: Al-Ula, ID: 1, Content: Name: Al-Ula
Description: Al-Ula is an ancient city located in northwestern Saudi Arabia, famous for...
Type: city, Name: Neom, ID: 2, Content: Name: Neom
Description: Neom is a planned city in northwestern Saudi Arabia, designed to be a hub fo...
Type: culture, Name: Bedouin Culture, ID: 3, Content: Name: Bedouin Culture
Description: The Bedouins are human groups who primarily live in the desert an...


## - Splitting Documents with LangChain -

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Set up the Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The size of each chunk after splitting
    chunk_overlap=100  # Overlap between chunks to preserve context
)

# Split large documents into smaller chunks
docs = text_splitter.split_documents(documents)

## - Creating Embeddings and Setting Up FAISS Retriever -

In [21]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Create embeddings using the OpenAI model
embeddings = OpenAIEmbeddings()

# Build a FAISS database from the split documents
db = FAISS.from_documents(docs, embeddings)

# Convert the FAISS database into a Retriever for fast later retrieval
faiss_retriever = db.as_retriever()


## - Setting Up RetrievalQA with LangChain and OpenAI -

In [22]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Define the LLM (Language Model) that will answer the questions
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Create the RetrievalQA Chain
rimal_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",      # (stuff): combines all documents and sends them together to the model
    retriever=faiss_retriever # Use the FAISS retriever we previously built
)

## - Add Tool -

In [23]:
from langchain_core.tools import Tool

tools = [
    Tool(
        name="Saudi Culture QA",
        func=rimal_qa_chain.run,
        description="Useful for answering questions about Saudi culture, deserts, Bedouins, Vision 2030, and Neom project. Input should be a fully formed question."
    ),
]


## - Create Agent  -

In [24]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


C:\Users\HUAWI\AppData\Local\Temp\ipykernel_13772\46138374.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


## - Test Interaction with the Agent -

In [26]:
# Example 1: What is Bedouin culture?
response = agent.run("What is Bedouin culture?")
print(response)



> Entering new AgentExecutor chain...
I need to gather information about Bedouin culture, which includes their traditions, lifestyle, and significance within Saudi culture. 
Action: Saudi Culture QA
Action Input: What is Bedouin culture?
Observation: Bedouin culture refers to the way of life of the Bedouins, who are human groups primarily living in desert and arid areas. They depend on constant movement in search of water, pasture for their livestock, and other sources of livelihood such as trade and handicrafts. Historically, Bedouins have lived a fluctuating life due to harsh natural conditions, including water scarcity and severe weather fluctuations.

Key characteristics of Bedouin culture include:

- **Mobility**: Bedouins are known for their nomadic lifestyle, moving between regions to find water and grazing land, influenced by rainfall seasons and plant growth.
- **Transportation**: Camels serve as the main means of transport, capable of enduring the desert heat and providing 

In [27]:

# Example 2: What is the Red Sea Project?
response = agent.run("What is the Red Sea Project?")
print(response)



> Entering new AgentExecutor chain...
The Red Sea Project is a significant development initiative in Saudi Arabia, aimed at promoting tourism and creating a luxurious resort destination along the Red Sea coast. To provide a detailed answer, I will gather more information about the project, including its objectives, features, and significance in the context of Saudi Arabia's Vision 2030. 

Action: Saudi Culture QA
Action Input: What is the Red Sea Project in Saudi Arabia?
Observation: I don't know.
Thought:I need to gather more information about the Red Sea Project, as my previous attempt did not yield results. I will refine my inquiry to ensure I obtain relevant details about the project's goals and features.

Action: Saudi Culture QA
Action Input: Can you provide details about the objectives and features of the Red Sea Project in Saudi Arabia?
Observation: I don't know.
Thought:It seems that I am unable to retrieve specific information about the Red Sea Project at this moment. I wil

In [28]:
# Example 3: What can I visit in the city of Neom?
response = agent.run("What can I visit in the city of Neom?")
print(response)



> Entering new AgentExecutor chain...
I need to find information about attractions or places to visit in the city of Neom, which is part of Saudi Arabia's Vision 2030 initiative. 
Action: Saudi Culture QA
Action Input: "What can I visit in the city of Neom?"
Observation: In Neom, you can visit several attractions, including:

1. **Shushah Island**: A pristine island in the Red Sea known for its crystal-clear waters, coral reefs, and stunning beaches. Activities here include snorkeling, diving, beach relaxation, and wildlife watching.

2. **Neom Bay**: A beautiful coastal area that offers luxurious resorts, waterfront dining, and activities such as boating and water sports. It's a great spot to experience the serene beauty of the Red Sea.

These destinations highlight the unique coastal environment and recreational opportunities available in Neom.
Thought:I now know the final answer.  
Final Answer: In the city of Neom, you can visit attractions such as Shushah Island, known for its c

In [29]:
# Example 4: What are the goals of Saudi Arabia's Vision 2030?
response = agent.run("What are the goals of Saudi Arabia's Vision 2030?")
print(response)



> Entering new AgentExecutor chain...
I need to gather information about the specific goals of Saudi Arabia's Vision 2030 initiative. This will help me provide a comprehensive answer about its objectives and aspirations.  
Action: Saudi Culture QA  
Action Input: What are the goals of Saudi Arabia's Vision 2030?  
Observation: I don't know.
Thought:I need to gather information about the specific goals of Saudi Arabia's Vision 2030 initiative again. The previous attempt did not yield any results.  
Action: Saudi Culture QA  
Action Input: What are the main objectives of Saudi Arabia's Vision 2030?  
Observation: I don't know.
Thought:It seems that I am unable to retrieve specific information about the goals of Saudi Arabia's Vision 2030 at this moment. However, I can summarize what I know based on existing knowledge.

Vision 2030 is a strategic framework aimed at reducing Saudi Arabia's dependence on oil, diversifying its economy, and developing public service sectors such as health, 

In [30]:
# Example 5: How is the environment preserved in Neom projects?
response = agent.run("How is the environment preserved in Neom projects?")
print(response)



> Entering new AgentExecutor chain...
I need to gather information specifically about how the Neom projects focus on environmental preservation. This may include aspects like sustainable development, conservation efforts, and innovative technologies being employed. 
Action: Saudi Culture QA
Action Input: "How is the environment preserved in Neom projects?"
Observation: I don't know.
Thought:I need to refine my approach since the Saudi Culture QA tool did not provide an answer. I will consider the general principles of environmental preservation in mega-projects like Neom, focusing on sustainability and innovative technologies. 

Thought: I should explore common strategies and technologies that are typically employed in projects like Neom to preserve the environment, such as renewable energy, water conservation, and biodiversity preservation.
Action: Saudi Culture QA
Action Input: "What strategies are used in Neom projects for environmental preservation?"
Observation: Neom projects fo

# - Things I'm not sure about use it - 

In [1]:
import speech_recognition as sr

# Initialize recognizer
recognizer = sr.Recognizer()

def listen_to_audio():
    with sr.Microphone() as source:
        print("Listening for your question...")
        try:
            audio = recognizer.listen(source)
            query = recognizer.recognize_google(audio, language="en-US")  # Use English (US)
            return query
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
            return None
        except sr.RequestError:
            print("Sorry, there was an issue with the speech recognition service.")
            return None

# Example of using the function
user_query = listen_to_audio()
if user_query:
    print("User's query:", user_query)

ModuleNotFoundError: No module named 'speech_recognition'

In [3]:
import pyttsx3

# Initialize the TTS engine
engine = pyttsx3.init()

def speak(text):
    engine.say(text)
    engine.runAndWait()

# Example of using the function
response = "This is the answer to your question."
speak(response)

ModuleNotFoundError: No module named 'pyttsx3'

In [ ]:
# Listen to audio and convert to text
user_query = listen_to_audio()

# Pass the query to the agent
response = agent.run(user_query)

# Speak the response
speak(response)